In [1]:
import torchvision
model = torchvision.models.optical_flow.raft_large(pretrained=True)

/home/wg25r/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/wg25r/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Raft_Large_Weights.C_T_SKHT_V2`. You can also use `weights=Raft_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import cv2
import numpy as np
import torch
import torchvision
import torchvision.transforms.functional
import torchvision.transforms.v2
import tqdm
model = model.cuda()
import os
os.makedirs("flow", exist_ok=True)

for video_name in tqdm.tqdm(os.listdir("COOOL Benchmark")):
    cap = cv2.VideoCapture(f"COOOL Benchmark/{video_name}")

    frame_width = 448
    frame_height = 448

    out = cv2.VideoWriter(f'flow/{video_name}.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

    last_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame1 = cv2.cvtColor(last_frame, cv2.COLOR_BGR2RGB).astype(np.float32)
        frame2 = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB).astype(np.float32)
        frame1 = cv2.resize(frame1, (frame_width, frame_height))
        frame2 = cv2.resize(frame2, (frame_width, frame_height))
        
        frame1 = torchvision.transforms.functional.normalize(torchvision.transforms.v2.ToImage()(frame1).unsqueeze(0), [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]).cuda()
        frame2 = torchvision.transforms.functional.normalize(torchvision.transforms.v2.ToImage()(frame2).unsqueeze(0), [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]).cuda()
        with torch.no_grad():
            flow = model(frame1, frame2)[0][-1] 
        flow_rgb = torchvision.utils.flow_to_image(flow).permute(1, 2, 0).cpu().numpy().astype(np.uint8)
        
        last_frame = frame.copy()
        out.write(flow_rgb) 

    cap.release()
    out.release()

 82%|████████▏ | 163/200 [1:23:26<16:51, 27.35s/it]